In [ ]:
import google.colab
import sqlite3
import contextlib
import os
import http.server
import urllib.parse
import html

PORT = 18080
ADDRESS = ("localhost", PORT)
DATAFILE = "test.db"

# データベース用意
if os.path.isfile(DATAFILE):
    os.remove(DATAFILE)
conn = sqlite3.connect(DATAFILE)
with contextlib.closing(conn.cursor()) as cur:
    cur.executescript(
        """
        CREATE TABLE post(id INTEGER PRIMARY KEY AUTOINCREMENT, name STRING, text STRING);
        INSERT INTO post(name, text) VALUES ("管理人","初めての投稿です");
        """
    )
    cur.connection.commit()


# Webアプリ本体
class MyHTTPRequestHandler(http.server.BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(200)
        self.send_header("Content-Type", "text/html; charset=utf-8")
        self.end_headers()

        response_html = """
        <html>
        <head>
        <meta charset="UTF-8">
        <title>掲示板 : Webアプリセキュリティ演習</title>
        </head>
        <body>
        <h1>掲示板</h1>
        """

        with contextlib.closing(conn.cursor()) as cur:
            cur.execute("SELECT name,text FROM post")
            data = cur.fetchall()
            for d in data:
                response_html += "<h3>" + d[0] + "</h3>"
                response_html += "<div>" + d[1] + "</div>"

        response_html += """
        <h1>新規投稿</h1>
        <form method="post">
            名前: <input type="text" name="name"><br>
            <textarea name="text"></textarea><br>
            <button type="submit">投稿</button>
        </form>
        </body>
        </html>
        """
        self.wfile.write(response_html.encode())

    def do_POST(self):
        length = int(self.headers.get("content-length"))
        field_data = self.rfile.read(length)
        fields = urllib.parse.parse_qs(field_data.decode("UTF-8"))
        name = ""
        if "name" in fields:
            name = fields["name"][0]
        text = ""
        if "text" in fields:
            text = fields["text"][0]
        with contextlib.closing(conn.cursor()) as cur:
            cur.executescript(
                'INSERT INTO post(name, text) VALUES ("' + name + '","' + text + '")'
            )
            cur.connection.commit()
        self.do_GET()


# 起動
with http.server.HTTPServer(ADDRESS, MyHTTPRequestHandler) as server:
    google.colab.output.serve_kernel_port_as_window(PORT, path="")
    server.serve_forever()

### 情報セキュリティ演習 - Webアプリケーション

- クロスサイトスクリプティング

```
<script>document.body.style.display='none'</script>
```

- SQLインジェクション

```
");
DELETE FROM post; SELECT
("
```

